This is the notebook of GNNs, I implement various GNN models here.

learning material: http://web.stanford.edu/class/cs224w/

ML for Graph don't need the feature engineering part. After the raw data is processed into graph data, the ML model will automacally learn the feature and do the downstream task.

Map nodes to d-dimensional embeddings such that similar nodes in the network are embedded close together.



In [1]:
import torch
from torch_geometric.nn import MessagePassing 
from torch_geometric.utils import add_self_loops, degree 

In [18]:
from typing import Callable, List, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch import Tensor
from torch.optim import Optimizer
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import tqdm


In [3]:
dataset = Planetoid(root='./cora', name='Cora')

Processing...
Done!


In [9]:
print(type(dataset))
print(dataset.num_classes)
print(dataset.num_node_features)

<class 'torch_geometric.datasets.planetoid.Planetoid'>
7
1433


In [12]:
class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        #二层卷积
        self.conv1 = GCNConv(dataset.num_node_features,16)
        self.conv2 = GCNConv(16,dataset.num_classes)
    def forward(self,data):
        x,edge_index = data.x,data.edge_index
        x = self.conv1(x,edge_index)
        x = F.relu(x)
        x = F.dropout(x,training=self.training)

        x = self.conv2(x,edge_index)
        return F.log_softmax(x,dim=1)

In [17]:
import torch.optim as optim
# gpu加速
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)

optimizer = optim.Adam(model.parameters(),lr=0.01,weight_decay=5e-4)
print(model)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)


In [20]:
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask],data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [21]:
model.eval()
pred = model(data).argmax(dim=1)
#print(pred)
#pred1 = pred[data.test_mask].cpu().numpy()
# data1 = data[data.test_mask].cpu().numpy()
#print(pred1)
#print(data.y[data.test_mask])
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()
print(correct)
acc = int(correct) / int(data.test_mask.sum())
print(acc)

808
0.808
